# Working with Geospatial Data


In this lesson, you work with ```JSON``` data accessed via the Colorado information warehouse. The data will contain geospatial information nested within it that will allow us to create a map of the data.


Check out the map [Colorado DWR Current Surface Water Conditions map](https://data.colorado.gov/Water/DWR-Current-Surface-Water-Conditions-Map-Statewide/j5pc-4t32).

Remember from the previous lesson, APIs can be used for many different things. Web developers (people who program and create web sites and cool applications) can use APIs to create user friendly interfaces - like the map in the previous example that allows us to look at and interact with data. These APIs are similar to, if not the same as, the ones that you often use to access data in ```Python```.

In this lesson, you will access the data used to create the map at the link above using ```Python```.

- The data that you will use are located here: [View JSON format data used to create surface water map](https://data.colorado.gov/resource/j5pc-4t32.json).
- And you can learn more about the data here: [View CO Current water surface](https://data.colorado.gov/Water/Current-Surface-Water-Conditions-in-Colorado/4yw9-a5y6).



In [1]:
import requests
import folium
import urllib
from pandas.io.json import json_normalize
import pandas as pd
import folium
from geopandas import GeoDataFrame
from shapely.geometry import Point

/Users/brpagan/anaconda3/envs/traillop/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/6f/styv5_3s4vz2g80lbzd67b8m0000gq/T/ipykernel_11012/3499686145.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely

In [2]:
water_base_url = "https://data.colorado.gov/resource/j5pc-4t32.json?"
water_full_url = water_base_url + "station_status=Active" + "&county=BOULDER"

In [3]:
water_full_url


'https://data.colorado.gov/resource/j5pc-4t32.json?station_status=Active&county=BOULDER'

In [4]:
data = requests.get(water_full_url)


In [5]:
type(data.json())


list

Remember that the JSON structure supports hierarchical data and can be NESTED. If you look at the structure of the ```.json``` file below, you can see that the location object, is nested with three sub objects:

- latitude
- longitude
- needs_recoding

Since ```data.json()``` is a ```list``` you can print out just the first few items of the list to look at your data as a sanity check.

In [6]:
data.json()[:2]


[{'station_name': 'WELLMAN DITCH TAIL',
  'div': '1',
  'location': {'latitude': '40.007326',
   'needs_recoding': False,
   'longitude': '-105.216119'},
  'dwr_abbrev': 'WLMTALCO',
  'data_source': 'Co. Division of Water Resources',
  'amount': '0.00',
  'station_type': 'Diversion',
  'wd': '6',
  'http_linkage': {'url': 'https://dwr.state.co.us/Tools/Stations/WLMTALCO'},
  'date_time': '2023-04-11T16:15:00.000',
  'county': 'BOULDER',
  'variable': 'DISCHRG',
  'station_status': 'Active'},
 {'station_name': 'NORTH BOULDER FARMERS DITCH RETURN TO BOULDER CREEK',
  'div': '1',
  'location': {'latitude': '40.01495',
   'needs_recoding': False,
   'longitude': '-105.278493'},
  'dwr_abbrev': 'NBFRTNCO',
  'data_source': 'Co. Division of Water Resources',
  'amount': '0.00',
  'station_type': 'Diversion',
  'wd': '6',
  'http_linkage': {'url': 'https://dwr.state.co.us/Tools/Stations/NBFRTNCO'},
  'date_time': '2023-04-11T15:15:00.000',
  'county': 'BOULDER',
  'variable': 'DISCHRG',
  'st

## Convert JSON to Pandas DataFrame
Now that you have pulled down the data from the website, you have it in the ```JSON``` format. For the next step, you will use the ```json_normalize()``` function from the Pandas library to convert this data into a ```Pandas``` ```DataFrame```.

This function helps organize and flatten data into a semi-structed table. To learn more, check out the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html)!

In [7]:
from pandas.io.json import json_normalize


In [8]:
result = json_normalize(data.json())


/var/folders/6f/styv5_3s4vz2g80lbzd67b8m0000gq/T/ipykernel_11012/3787952420.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  result = json_normalize(data.json())


In [9]:
result.head()


,station_name,div,dwr_abbrev,data_source,amount,station_type,wd,date_time,county,variable,station_status,location.latitude,location.needs_recoding,location.longitude,http_linkage.url,usgs_station_id,stage,flag
0,WELLMAN DITCH TAIL,1,WLMTALCO,Co. Division of Water Resources,0.00,Diversion,6,2023-04-11T16:15:00.000,BOULDER,DISCHRG,Active,40.007326,False,-105.216119,https://dwr.state.co.us/Tools/Stations/WLMTALCO,NaN,NaN,NaN
1,NORTH BOULDER FARMERS DITCH RETURN TO BOULDER ...,1,NBFRTNCO,Co. Division of Water Resources,0.00,Diversion,6,2023-04-11T15:15:00.000,BOULDER,DISCHRG,Active,40.01495,False,-105.278493,https://dwr.state.co.us/Tools/Stations/NBFRTNCO,NaN,NaN,NaN
2,"LEFT HAND CREEK NEAR BOULDER, CO.",1,LEFCRECO,Co. Division of Water Resources,8.72,Stream,5,2023-04-11T16:15:00.000,BOULDER,DISCHRG,Active,40.126407,False,-105.30451,https://dwr.state.co.us/Tools/Stations/LEFCRECO,06724500,0.35,NaN
3,BONUS DITCH,1,BONDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.14,Diversion,5,2023-04-11T16:00:00.000,BOULDER,DISCHRG,Active,40.153369,False,-105.088705,https://dwr.state.co.us/Tools/Stations/BONDITCO,NaN,0.05,NaN
4,WEBSTER MCCASLIN DITCH,1,WEBDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.26,Diversion,5,2023-04-11T16:00:00.000,BOULDER,DISCHRG,Active,40.193287,False,-105.210434,https://dwr.state.co.us/Tools/Stations/WEBDITCO,NaN,0.13,NaN


In [10]:
type(result)


pandas.core.frame.DataFrame

In [11]:
result.columns


Index(['station_name', 'div', 'dwr_abbrev', 'data_source', 'amount',
       'station_type', 'wd', 'date_time', 'county', 'variable',
       'station_status', 'location.latitude', 'location.needs_recoding',
       'location.longitude', 'http_linkage.url', 'usgs_station_id', 'stage',
       'flag'],
      dtype='object')

## Data Cleaning for Visualization
Now you can clean up the data. Notice that your longitude and latitude values are stored as strings. Do you think can create a map if these values are stored as strings?



In [12]:
result['location.latitude'][0]


'40.007326'

You can convert the strings to type float as follows.



In [13]:
result['location.latitude'] = result['location.latitude'].astype(float)


In [14]:
result['location.latitude'][0]


40.007326

In [15]:
result['location.longitude'] = result['location.longitude'].astype(float)


In [16]:
result['location.longitude'][0]


-105.216119

Now that you have numeric values for mapping, make sure that are are no missing values.



In [17]:
result.shape


(107, 18)

In [18]:
result['location.longitude'].isna().any()


False

In [19]:
result['location.latitude'].isna().any()


False

There are no ```nan``` values in this data. However, if there were, you could remove rows where a column has a nan value in a specific column with the following: ```result_nonan = result.dropna(subset=['location.longitude', 'location.latitude'])```

## Data Visualization
You will use the folium package to visualize the data. One approach you could take would be to convert your Pandas DataFrame to a Geopandas DataFrame for easy mapping.

In [20]:
geometry = [Point(xy) for xy in zip(result['location.longitude'], result['location.latitude'])]
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(result, crs=crs, geometry=geometry)
gdf.head()


/Users/brpagan/anaconda3/envs/traillop/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,station_name,div,dwr_abbrev,data_source,amount,station_type,wd,date_time,county,variable,station_status,location.latitude,location.needs_recoding,location.longitude,http_linkage.url,usgs_station_id,stage,flag,geometry
0,WELLMAN DITCH TAIL,1,WLMTALCO,Co. Division of Water Resources,0.00,Diversion,6,2023-04-11T16:15:00.000,BOULDER,DISCHRG,Active,40.007326,False,-105.216119,https://dwr.state.co.us/Tools/Stations/WLMTALCO,NaN,NaN,NaN,POINT (-105.21612 40.00733)
1,NORTH BOULDER FARMERS DITCH RETURN TO BOULDER ...,1,NBFRTNCO,Co. Division of Water Resources,0.00,Diversion,6,2023-04-11T15:15:00.000,BOULDER,DISCHRG,Active,40.014950,False,-105.278493,https://dwr.state.co.us/Tools/Stations/NBFRTNCO,NaN,NaN,NaN,POINT (-105.27849 40.01495)
2,"LEFT HAND CREEK NEAR BOULDER, CO.",1,LEFCRECO,Co. Division of Water Resources,8.72,Stream,5,2023-04-11T16:15:00.000,BOULDER,DISCHRG,Active,40.126407,False,-105.304510,https://dwr.state.co.us/Tools/Stations/LEFCRECO,06724500,0.35,NaN,POINT (-105.30451 40.12641)
3,BONUS DITCH,1,BONDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.14,Diversion,5,2023-04-11T16:00:00.000,BOULDER,DISCHRG,Active,40.153369,False,-105.088705,https://dwr.state.co.us/Tools/Stations/BONDITCO,NaN,0.05,NaN,POINT (-105.08871 40.15337)
4,WEBSTER MCCASLIN DITCH,1,WEBDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.26,Diversion,5,2023-04-11T16:00:00.000,BOULDER,DISCHRG,Active,40.193287,False,-105.210434,https://dwr.state.co.us/Tools/Stations/WEBDITCO,NaN,0.13,NaN,POINT (-105.21043 40.19329)


Then, you can plot the data using the folium functions ```GeoJson()``` and ```add_to()``` to add the data from the ```Geopandas``` ```DataFrame``` to the map object

In [21]:
m = folium.Map([40.01, -105.27], zoom_start= 10, tiles='cartodbpositron')
folium.GeoJson(gdf).add_to(m)

m

Great! You now have an interactive map in your notebook!

You can also cluster the markers, and add a popup to each marker, so you can give your viewers more information about station: such as its name and the amount of precipitation measured.

For this example below, you will work with the ```Pandas``` ```DataFrame``` you originally created from the ```JSON```, instead of the ```Geopandas GeoDataFrame```.

In [22]:
# Get the latitude and longitude from result as a list
locations = result[['location.latitude', 'location.longitude']]
coords = locations.values.tolist()

In [23]:
from folium.plugins import MarkerCluster

m = folium.Map([40.01, -105.27], zoom_start= 10, tiles='cartodbpositron')

marker_cluster = MarkerCluster().add_to(m)

for point in range(0, len(coords)):
    folium.Marker(location = coords[point], popup= 'Name: ' + result['station_name'][point] + ' ' + 'Precip: ' + str(result['amount'][point])).add_to(marker_cluster)

m